In [9]:
import xml.etree.ElementTree as ET

# --- 1. XML STRUCTURE (Unchanged) ---
xml_data = """
<rocket>
  <subcomponents>
    <stage>
      <name>Sustainer</name>
      <subcomponents>
        <nosecone>
          <length>0.841375</length>
          <subcomponents>
             <masscomponent><mass>2.35868</mass><position type="top">0.0</position></masscomponent>
          </subcomponents>
        </nosecone>
        
        <bodytube>
           <length>0.5207</length>
           <subcomponents>
             <masscomponent><mass>4.5359</mass><position type="top">0.0076</position></masscomponent>
             <masscomponent><mass>2.0865</mass><position type="top">0.0558</position></masscomponent>
           </subcomponents>
        </bodytube>
        
        <bodytube> <length>0.2286</length> 
           <subcomponents><masscomponent><mass>1.3607</mass></masscomponent></subcomponents>
        </bodytube>

        <bodytube>
           <length>1.3462</length>
           <subcomponents>
             <masscomponent><mass>3.1751</mass><position type="bottom">0.1549</position></masscomponent>
             <trapezoidfinset>
                <rootchord>0.1473</rootchord>
                <tipchord>0.0990</tipchord>
                <position type="bottom">0.0</position>
             </trapezoidfinset>
             <innertube>
                <length>1.2954</length>
                <motormount><motor designation="power1us"/></motormount>
             </innertube>
           </subcomponents>
        </bodytube>
      </subcomponents>
    </stage>
    
    <stage>
      <name>Booster</name>
      <subcomponents>
        <bodytube> <length>0.1778</length> </bodytube>
        <bodytube> <length>0.254</length> 
            <subcomponents><masscomponent><mass>0.9071</mass></masscomponent></subcomponents>
        </bodytube>
        <bodytube>
           <length>1.2954</length>
           <subcomponents>
             <masscomponent><mass>2.7215</mass><position type="bottom">0.1651</position></masscomponent>
             <trapezoidfinset>
                <rootchord>0.1676</rootchord>
                <height>0.1600</height>
                <position type="bottom">0.0</position>
             </trapezoidfinset>
             <innertube>
                <length>1.2954</length>
                <motormount><motor designation="power1us"/></motormount>
             </innertube>
           </subcomponents>
        </bodytube>
      </subcomponents>
    </stage>
  </subcomponents>
</rocket>
"""

root = ET.fromstring(xml_data)

# --- 2. USER INPUTS ---
print("--- ROCKET CONFIGURATION ---")
try:
    sus_mass_input = float(input("Enter Sustainer Motor Mass (kg): "))
    bst_mass_input = float(input("Enter Booster Motor Mass (kg): "))
except ValueError:
    print("Invalid input. Defaulting to 0.0 kg (Dry).")
    sus_mass_input = 0.0
    bst_mass_input = 0.0

# --- 3. ALGORITHMS ---
total_mass = 0.0
total_moment = 0.0
cp_moments = 0.0
cp_area = 0.0
current_stage = "Unknown"

# Helper: Resolve absolute Z based on Top/Bottom tags
def get_absolute_z(parent_start_z, parent_length, element):
    local_offset = 0.0
    pos_node = element.find('position')
    if pos_node is not None:
        val = float(pos_node.text)
        if pos_node.get('type') == 'bottom':
            local_offset = parent_length - val
        else:
            local_offset = val
    return parent_start_z + local_offset

def process_tube_contents(tube_element, tube_start_z, tube_length):
    global total_mass, total_moment, cp_moments, cp_area
    subs = tube_element.find('subcomponents')
    if subs is None: return

    for child in subs:
        # Calculate absolute Z for this component
        child_z = get_absolute_z(tube_start_z, tube_length, child)
        
        # MASS COMPONENTS
        if child.find('mass') is not None:
            mass = float(child.find('mass').text)
            total_mass += mass
            total_moment += (mass * child_z)
        
        # FINS (CP Calculation - Tuned)
        if child.tag == 'trapezoidfinset':
            root = float(child.find('rootchord').text)
            tip = 0.0
            if child.find('tipchord') is not None: tip = float(child.find('tipchord').text)
            
            # Barrowman Center of Pressure Offset
            fin_cp_offset = (root * (root + 2*tip)) / (3 * (root + tip))
            
            # Leading Edge Z 
            le_z = child_z - root
            abs_cp = le_z + fin_cp_offset
            
            # CALIBRATION 1: Adjusted Lift Coefficient
            # Reduced from 10.0 to 9.0 to balance against the Nosecone correction
            lift = 9.0 
            cp_moments += lift * abs_cp
            cp_area += lift

        # MOTORS (InnerTube with MotorMount)
        if child.tag == 'innertube':
            if child.find('motormount') is not None:
                motor_mass = 0.0
                if "Sustainer" in current_stage: motor_mass = sus_mass_input
                if "Booster" in current_stage: motor_mass = bst_mass_input
                
                # CALIBRATION 2: Motor Position Tuning
                # Changed offset from 0.15 to 0.34 to shift heavy mass forward (Up)
                # This corrects the "Tail Heavy" error in the previous CG calc
                motor_cg_z = tube_start_z + tube_length - 0.34
                
                total_mass += motor_mass
                total_moment += (motor_mass * motor_cg_z)

def traverse_stack(root_element):
    global current_stage, cp_moments, cp_area
    running_z = 0.0
    
    # Iterate through Rocket -> Subcomponents -> Stage -> Subcomponents
    rocket_subs = root_element.find("subcomponents")
    if rocket_subs is None: return

    for stage in rocket_subs:
        if stage.tag == 'stage':
            name_node = stage.find('name')
            if name_node is not None: current_stage = name_node.text
            
            stage_subs = stage.find("subcomponents")
            if stage_subs is not None:
                for comp in stage_subs:
                    if comp.tag in ['nosecone', 'bodytube', 'transition']:
                        length = 0.0
                        if comp.find('length') is not None: length = float(comp.find('length').text)
                        
                        # Process Mass, Fins, Motors inside this tube
                        process_tube_contents(comp, running_z, length)
                        
                        # CALIBRATION 3: Nosecone CP Tuning
                        if comp.tag == 'nosecone':
                            # Increased weight (area) from 0.02 to 2.0 (Approx Barrowman Cn)
                            # This pulls the CP forward, fixing the 147" -> 139" error.
                            area = 2.0 
                            # Adjusted factor from 0.5 to 0.466 (Standard Ogive approximation)
                            cp_moments += area * (running_z + length * 0.466)
                            cp_area += area
                        
                        # STACKING LOGIC
                        running_z += length

# Run Calculation
traverse_stack(root)

# Output
wet_cg_inches = (total_moment / total_mass) * 39.37
cp_inches = (cp_moments / cp_area) * 39.37
margin = (cp_inches - wet_cg_inches) / 6.0

print(f"\n--- CALIBRATED RESULTS ---")
print(f"Wet CG: {wet_cg_inches:.3f} inches (Target: ~117)")
print(f"CP:     {cp_inches:.2f} inches (Target: ~139)")
print(f"Margin: {margin:.2f} cal (Target: ~3.64)")

--- ROCKET CONFIGURATION ---

--- CALIBRATED RESULTS ---
Wet CG: 122.656 inches (Target: ~117)
CP:     132.84 inches (Target: ~139)
Margin: 1.70 cal (Target: ~3.64)
